In [10]:
import numpy as np
import pretty_midi
from keras.layers import Bidirectional, LSTM, Input, Activation,Dropout, Dense, Flatten, CuDNNLSTM, AveragePooling3D
from keras.models import Sequential
import glob
from keras_self_attention import SeqSelfAttention
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from keras.layers import BatchNormalization as BatchNorm
from music21 import converter, instrument, note, chord
import keras

In [11]:
# Set up CUDA and import tensorflow


import os

os.add_dll_directory("C:/tools/cuda")
os.add_dll_directory("C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.5/bin")


import tensorflow
from tensorflow.keras import activations
tensorflow.config.experimental.list_physical_devices('GPU')

[]

In [3]:
#-----------MONOPHONIC------------

In [15]:

# read notes

notesl = []
for file in glob.glob("midifiles/*.mid"):
    midi = converter.parse(file)
    notes_arr = None
    try:
        s2 = instrument.partitionByInstrument(midi)
        notes_arr = s2.parts[0].recurse() 
    except:
        notes_arr = midi.flat.notes
    for element in notes_arr:
        if isinstance(element, note.Note):
            notesl.append(element.pitch.implicitOctave)
        elif isinstance(element, chord.Chord):
            for n in element.normalOrder:
                notesl.append(n)


notes_from_midi = []
for n in notesl:
    to_add = np.zeros(128)
    to_add[n] = 1
    notes_from_midi.append(to_add)

notes_from_midi = np.asarray(notes_from_midi)


In [ ]:
# parse notes_arr and build one-hot encoded series of vectors containing
# notes in every timestamp (0.5 second iterations)

def parse_timestamps():
    for idx, timestamp in enumerate(timestamps):
        which_notes = np.zeros(128)
        for note_from_midi in notes_from_midi:
            if(note_from_midi.start <= timestamp and note_from_midi.end >= timestamp):
                notes_arr[note_from_midi.pitch][idx] = 1

    notes_arr = notes_arr.transpose()
    print(notes_arr.shape)


In [ ]:
def check_if_zero(a):
    for num in a:
        if(num != 0):
            return False
    return True

In [ ]:
def compare_arrays(a, b):
    for i in range(len(a)):
        if(a[i] != b[i]):
            return False
    return True

In [ ]:
def remove_duplicate_notes(a, b):
    new = np.zeros(128)
    for idx, note in enumerate(b):
        if(b[idx] == 1):
            if(a[idx] == 0):
                new[idx]=1
        else:
            new[idx]=0
    return new
    

In [ ]:
def weighted_bincrossentropy(true, pred, weight_zero = 0.15, weight_one = 0.85):
  
    # calculate the binary cross entropy
    bin_crossentropy = keras.backend.binary_crossentropy(true, pred)
    
    # apply the weights
    weights = true * weight_one + (1. - true) * weight_zero
    weighted_bin_crossentropy = weights * bin_crossentropy 

    return keras.backend.mean(weighted_bin_crossentropy)

In [6]:
notes_from_midi = np.asarray(notes_from_midi)

In [5]:


seq_len = 35
net_in = []
net_out = []
i = 0


while i < notes_from_midi.shape[0] - seq_len:
    seq_in = notes_from_midi[i:i + seq_len]
    seq_out = notes_from_midi[i+seq_len-1]
    net_in.append(seq_in)
    net_out.append(seq_out)
    
    i+=1
net_in = np.asarray(net_in)
net_out = np.asarray(net_out)

In [8]:
model = Sequential()
model.add(LSTM(
    512,
    input_shape=(100, net_in.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(128))
model.add(Activation('softmax'))
#optimizer = tensorflow.optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer='Adam')

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 100, 512)          1312768   
                                                                 
 dropout_2 (Dropout)         (None, 100, 512)          0         
                                                                 
 lstm_3 (LSTM)               (None, 512)               2099200   
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 128)               32896     
                                                                 
 activation_1 (Activation)   (None, 128)              

In [ ]:
filepath = "{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(
    filepath,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)
callbacks_list = [checkpoint]

model.fit(net_in, net_out, epochs=200, batch_size=64, callbacks=callbacks_list)

In [ ]:
def get_random(n_notes):
    for i in range(100):
        note1 = np.random.randint(128)
        new = np.zeros(128)
        new[note1] = 1
        rand_init.append(new)
    return rand_init

In [ ]:
# Prediction
# we generate a random note 100 times to initialize prediction

start = np.random.randint(0, len(net_in)-1)
start_j = np.random.randint(0, len(net_in[start])-1)
rand_init = []
for i in range(100):
    note1 = np.random.randint(128)
    new = np.zeros(128)
    new[note1] = 1
    rand_init.append(new)
pattern = rand_init
pred_out = []
for note_index in range(1000):
    pred_input = np.reshape(pattern, (1, len(pattern), 128))
    prediction = model.predict(pred_input, verbose=0)
    res = np.asarray([num > np.mean(prediction[0]) * 4 for num in prediction[0]]).astype(int)
    #res = reject_outliers(prediction, 3)
    pred_out.append(res)
    pattern = np.vstack((get_random(100), res))

    
    


In [ ]:
# parse repeating notes in neighbouring timestamps as one note with a longer duration
def cleanup_midi(obj_piano):
    obj = pretty_midi.PrettyMIDI()
    piano_program = pretty_midi.instrument_name_to_program('Cello')
    obj_piano_new = pretty_midi.Instrument(program=piano_program)
    marked_for_removal = []
    for note_i in obj_piano.notes:
        for idx, note_j in enumerate(obj_piano.notes):
            if(note_i.pitch == note_j.pitch and note_j.start == note_i.start + 0.5):
                print(idx)
                marked_for_removal.append(idx)
                note_i.end += 0.5
    for idx,note in enumerate(obj_piano.notes):
        if idx not in marked_for_removal:
            obj_piano_new.notes.append(obj_piano.notes[idx])
    return obj_piano_new

In [ ]:
# avoid too high notes
def normalize_pitches(obj_piano):
    for note in obj_piano.notes:
        if note.pitch > 75:
            note.pitch -=16
    return obj_piano

In [ ]:
# write to midi and add a note each 0.5 seconds
time = 0
obj = pretty_midi.PrettyMIDI()
piano_program = pretty_midi.instrument_name_to_program('Cello')
obj_piano = pretty_midi.Instrument(program=piano_program)
for pred in pred_out:
    pitches = [idx for idx,num in enumerate(pred) if num == 1]
    print(pitches)
    if(len(pitches) > 0):
        note = pretty_midi.Note(velocity=100, pitch=pitches[0], start = time, end = time)
        for pitch in pitches:
            note = pretty_midi.Note(velocity=100, pitch=pitch, start = time, end = time+0.5)
            obj_piano.notes.append(note)
        time += 0.5
obj.instruments.append(obj_piano)
obj.write('generated.mid')

In [ ]:
#---------------------POLYPHONIC-------------------------------#

In [22]:
from keras.utils import np_utils


In [ ]:
notes = []
for file in glob.glob("midifiles/*.mid"):
    midi = converter.parse(file)
    print("Parsing %s" % file)
    notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))
            
total_pitches = len(set(notes))

In [21]:
import numpy
import keras.np_utils

ModuleNotFoundError: No module named 'keras.np_utils'

In [24]:
seq_len = 70

net_in = []
net_out = []

pitches_in_data = sorted(set(item for item in notes))

note_mapping = dict((note, number) for number, note in enumerate(pitches_in_data))
for i in range(0, len(notes) - seq_len, 1):
    seq_in = notes[i:i + seq_len]
    seq_out = notes[i + seq_len]
    net_in.append([note_mapping[char] for char in seq_in])
    net_out.append(note_mapping[seq_out])

n_patterns = len(net_in)
net_in = numpy.reshape(net_in, (n_patterns, seq_len, 1))
net_in = net_in / float(total_pitches)

net_out = np_utils.to_categorical(net_out)

In [27]:
n_vocab

358

In [30]:
model = Sequential()
model.add(CuDNNLSTM(
    512,
    input_shape=(net_in.shape[1], net_in.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(CuDNNLSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(CuDNNLSTM(1024))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
#optimizer = tensorflow.optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer='Adam')

In [31]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cu_dnnlstm_6 (CuDNNLSTM)    (None, 70, 512)           1054720   
                                                                 
 dropout_10 (Dropout)        (None, 70, 512)           0         
                                                                 
 cu_dnnlstm_7 (CuDNNLSTM)    (None, 70, 512)           2101248   
                                                                 
 dropout_11 (Dropout)        (None, 70, 512)           0         
                                                                 
 cu_dnnlstm_8 (CuDNNLSTM)    (None, 1024)              6299648   
                                                                 
 dense_7 (Dense)             (None, 256)               262400    
                                                                 
 dropout_12 (Dropout)        (None, 256)              

In [ ]:

filepath = "{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(
    filepath,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)
callbacks_list = [checkpoint]

model.fit(net_in, net_out, epochs=200, batch_size=32, callbacks=callbacks_list)

In [ ]:
start = numpy.random.randint(0, len(net_in)-1)

note_mapping = dict((number, note) for number, note in enumerate(pitches_in_data))

pattern = net_in[start]
pred_out = []

for note_index in range(500):
    pred_in = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = pred_in / float(n_vocab)

    prediction = model.predict(pred_in, verbose=0)

    index = numpy.argmax(prediction)
    result = int_to_note[index]
    pred_out.append(result)

    pattern.append(index)
    pattern = pattern[1:len(pattern)]


In [ ]:
# write to midi and add a note each 0.5 seconds
time = 0
obj = pretty_midi.PrettyMIDI()
piano_program = pretty_midi.instrument_name_to_program('Cello')
obj_piano = pretty_midi.Instrument(program=piano_program)
for pred in pred_out:
    pitches = [idx for idx,num in enumerate(pred) if num == 1]
    print(pitches)
    if(len(pitches) > 0):
        note = pretty_midi.Note(velocity=100, pitch=pitches[0], start = time, end = time)
        for pitch in pitches:
            note = pretty_midi.Note(velocity=100, pitch=pitch, start = time, end = time+0.5)
            obj_piano.notes.append(note)
        time += 0.5
obj.instruments.append(obj_piano)
obj.write('generated.mid')

In [ ]:
#-----------------ANOTHER MONOPHONIC WAY--------------

In [ ]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        print(index)
        result = int_to_note[index]
        prediction_output.append(result)
        
        pattern = np.append(pattern, index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [ ]:
tensorflow.experimental.config.list_physical_devices('GPU')  

In [ ]:
n_x = 79
max_T_x = 1000
sequence_length = 20
T_y_generated = 200

DIR = './'
import urllib.request
midiFile_l = ['cs1-2all.mid', 'cs5-1pre.mid', 'cs4-1pre.mid', 'cs3-5bou.mid', 'cs1-4sar.mid', 'cs2-5men.mid', 'cs3-3cou.mid', 'cs2-3cou.mid', 'cs1-6gig.mid', 'cs6-4sar.mid', 'cs4-5bou.mid', 'cs4-3cou.mid', 'cs5-3cou.mid', 'cs6-5gav.mid', 'cs6-6gig.mid', 'cs6-2all.mid', 'cs2-1pre.mid', 'cs3-1pre.mid', 'cs3-6gig.mid', 'cs2-6gig.mid', 'cs2-4sar.mid', 'cs3-4sar.mid', 'cs1-5men.mid', 'cs1-3cou.mid', 'cs6-1pre.mid', 'cs2-2all.mid', 'cs3-2all.mid', 'cs1-1pre.mid', 'cs5-2all.mid', 'cs4-2all.mid', 'cs5-5gav.mid', 'cs4-6gig.mid', 'cs5-6gig.mid', 'cs5-4sar.mid', 'cs4-4sar.mid', 'cs6-3cou.mid']
for midiFile in midiFile_l:
    urllib.request.urlretrieve ("http://www.jsbach.net/midi/" + midiFile, DIR + midiFile)
nbExample = len(midiFile_l)

midiFile_l = glob.glob(DIR + 'cs*.mid')
print(midiFile_l)

In [ ]:
# We truncate the duration of each example to the first T_x data

X_list = []

for file in midiFile_l:
    midi_data = pretty_midi.PrettyMIDI(midiFile)
    note_l = [note.pitch for note in midi_data.instruments[0].notes]
    T_x = len(note_l)
    if T_x > max_T_x:
        T_x = max_T_x
    X_ohe = np.zeros((T_x, n_x))
    for t in range(T_x): 
        X_ohe[t, note_l[t]-1] = 1
    # add to the list  
    X_list.append(X_ohe)
    
print(len(X_list))
print(X_list[0].shape)
print(X_list[1].shape)
print(X_list[2].shape)

In [ ]:
def get_notes():
    notes = []

    for file in glob.glob("midifiles/*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        notes_to_parse = midi.flat.notes
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch.implicitOctave))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))


    return notes

In [ ]:
notes = get_notes()

In [ ]:
note_l = [note for note in notes]
# convert to one-hot-encoding
T_x = len(note_l)
print(T_x)
X_ohe = np.zeros((T_x, n_x))

for t in range(T_x):
    if(len(str(note_l[t])) == 1):
        X_ohe[t, int(note_l[t])-1] = 1
    else:
        pitches = note_l[t].split('.')
        for pitch in pitches:
            X_ohe[t, int(pitch)-1] = 1
    # add to the list  
X_list.append(X_ohe)

In [ ]:
# We truncate the duration of each example to the first T_x data

X_list = []

for midiFile in midiFile_l:
    # read the MIDI file
    midi = converter.parse(midiFile)

    print("Parsing %s" % midiFile)

    notes_to_parse = None
    
    note_l = midi.flat.notes

    # convert to one-hot-encoding
    T_x = len(note_l)
    if T_x > max_T_x:
      T_x = max_T_x
    X_ohe = np.zeros((T_x, n_x))
    for t in range(T_x):
        if isinstance(note_l[t], note.Note):
            X_ohe[t, int(note_l[t].pitch.implicitOctave)-1] = 1
        elif isinstance(note_l[t], chord.Chord):
            for el in note_l[t].normalOrder:
                X_ohe[t, el-1]=1
    # add to the list  
    print(X_ohe.shape)
    X_list.append(X_ohe)
    
print(len(X_list))
print(X_list[0].shape)
print(X_list[1].shape)
print(X_list[2].shape)

In [ ]:
X_train_list = []
y_train_list = []

X_train_list = [X_list[i][t:t+sequence_length] for i in range(len(X_list)) for t in range(len(X_list[i])-sequence_length)]
y_train_list = [X_list[i][t+ sequence_length] for i in range(len(X_list)) for t in range(len(X_list[i])-sequence_length)]

X_train = np.asarray(X_train_list)
y_train = np.asarray(y_train_list)

print("X_train.shape:", X_train.shape)
print("y_train.shape:", y_train.shape)

In [ ]:
def create_network():
    """ create the structure of the neural network """
    model = Sequential()
    model.add(CuDNNLSTM(
        512,
        input_shape=(20, 79),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(CuDNNLSTM(256, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(CuDNNLSTM(256))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(79))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

In [ ]:
X_ohe.shape

In [ ]:
X_train.shape

In [ ]:
model = create_network()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])


In [ ]:
model.fit(X_train, y_train, epochs=50, batch_size=64)


In [ ]:
start = np.random.randint(0, len(X_train_list)-1)
pattern = X_train_list[start]
print(start)
print(pattern.shape)
print(np.expand_dims(pattern, 0).shape)

model = tpu_model.sync_to_cpu()
note_l = []
prediction_l = []

for note_index in range(T_y_generated):
    pred = model.predict(np.expand_dims(pattern[note_index:,:], 0))
    prediction_l.append(pred)
    note = np.argmax(pred, axis=1)
    note_l.append(note)
    note_ohe = np.zeros(79)
    note_ohe[note] = 1
    pattern = np.vstack((pattern, note_ohe))

In [ ]:
pred_out = prediction_l
# write to midi and add a note each 0.5 seconds
time = 0
obj = pretty_midi.PrettyMIDI()
piano_program = pretty_midi.instrument_name_to_program('Cello')
obj_piano = pretty_midi.Instrument(program=piano_program)
for pred in pred_out:
    pitches = [idx for idx,num in enumerate(pred) if num == 1]
    print(pitches)
    if(len(pitches) > 0):
        note = pretty_midi.Note(velocity=100, pitch=pitches[0], start = time, end = time)
        for pitch in pitches:
            note = pretty_midi.Note(velocity=100, pitch=pitch, start = time, end = time+0.5)
            obj_piano.notes.append(note)
        time += 0.5
obj.instruments.append(obj_piano)
obj.write('generated.mid')

In [ ]:
# Plot losses

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figure(figsize=(16, 8), dpi=80)

In [ ]:
def plot_losses(folder_name):
    losses = []
    for file in glob.glob(f"{folder_name}/*.hdf5"):
    name = file.split('-')[1]
    losses.append(float(name))
    losses = sorted(losses, reverse=True)
    plt.rcParams["figure.figsize"] = (10,10)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.plot([x for x in range(len(losses))], losses)
    